# Comparing high-dimensional representations

Cross-validated methods to separate high-dimensional signal from noise

Especially after the advent of high-dimensional neural network models of
the brain, there has been an explosion of methods to compare
high-dimensional representations, including various forms of linear
regression, canonical correlation analysis (CCA), centered kernel
alignment (CKA), and non-linear methods too! In this part of the
tutorial, we’ll describe PLS-SVD – a method closely related to PCA –
that allows us to measure the similarity between two high-dimensional
systems in a manner that is relatively interpretable and extensible for
various purposes.

## PLS-SVD

Just as PCA identifies the principal directions of variance of a system,
PLS-SVD identifies the principal directions of *shared* variance between
*two* systems. Specifically, just as PCA computes the eigendecomposition
of the auto-covariance, PLS-SVD computes the singular value
decomposition of the cross-covariance:

$X^\top Y = U \Sigma V^\top$.

Here, the left singular vectors $U$ define a rotation of the system $X$
into some latent space, the right singular vectors $V$ define a rotation
of system $Y$ into the same latent space, and the singular values
$\Sigma$

> **What happens when $X$ = $Y$?**
>
> Note that if $X = Y$, PLS-SVD reduces to PCA:

In [ ]:
# TODO uncomment before final packaging
# %pip install git+https://github.com/BonnerLab/ccn-tutorial.git


In [ ]:
from collections.abc import Sequence
import warnings

import numpy as np
import pandas as pd
import xarray as xr
import torch
from sklearn.decomposition import TruncatedSVD
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
from IPython.display import display

from utilities.brain import (
    load_dataset,
    average_data_across_repetitions,
    load_stimuli,
    plot_brain_map,
)
from utilities.computation import svd


In [ ]:
%matplotlib inline

sns.set_theme(
    context="notebook",
    style="white",
    palette="deep",
    rc={"legend.edgecolor": "None"},
)
set_matplotlib_formats("svg")

pd.set_option("display.max_rows", 5)
pd.set_option("display.max_columns", 10)
pd.set_option("display.precision", 3)
pd.set_option("display.show_dimensions", False)

xr.set_options(display_max_rows=3, display_expand_data=False)

warnings.filterwarnings("ignore")

In [ ]:
random_state = 0
rng = np.random.default_rng(seed=random_state)


In [ ]:
class PLSSVD:
    def __init__(self) -> None:
        self.left_mean: np.ndarray
        self.right_mean: np.ndarray
        self.left_singular_vectors: np.ndarray
        self.right_singular_vectors: np.ndarray

    def fit(self, /, x: np.ndarray, y: np.ndarray) -> None:
        self.left_mean = x.mean(axis=-2)
        self.right_mean = y.mean(axis=-2)

        x_centered = x - self.left_mean
        y_centered = y - self.right_mean

        n_stimuli = x.shape[-2]

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        cross_covariance = (np.swapaxes(x_centered, -1, -2) @ y_centered) / (
            n_stimuli - 1
        )

        (
            self.left_singular_vectors,
            self.singular_values,
            self.right_singular_vectors,
        ) = svd(
            torch.from_numpy(cross_covariance).to(device),
            n_components=min([*x.shape, *y.shape]),
            truncated=True,
            seed=random_state,
        )

        n_stimuli = data.shape[-2]

        self.left_singular_vectors = self.left_singular_vectors.cpu().numpy()
        self.singular_values = self.singular_values.cpu().numpy()
        self.right_singular_vectors = self.right_singular_vectors.cpu().numpy()

    def transform(self, /, z: np.ndarray, *, direction: str) -> np.ndarray:
        match direction:
            case "left":
                return (z - self.left_mean) @ self.left_singular_vectors
            case "right":
                return (z - self.right_mean) @ self.right_singular_vectors
            case _:
                raise ValueError("direction must be 'left' or 'right'")


## Comparing brains

In the same way that we can PLS-SVD to estimate the shared variance
across presentations of the same stimuli *within* a participant, we can
also use it to estimate the shared variance in the neural
representations of the same stimuli *across* participants.

We have two data matrices $X \in \mathbb{R}^{N \times P_X}$ and
$Y \in \mathbb{R}^{N \times P_Y}$ from two participants. We could
directly compute the singular values of their cross-covariance, which
would be a direct estimate of the shared variance between these two
representations.

However, we run into the same issue as before: the singular values of a
matrix are always positive and we won’t be able to use the magnitude of
the singular value to assess the reliability of the variance along that
dimension.

Instead, we can use a cross-validated approach similar to CV-PCA, except
that instead of testing generalization across different *presentations*
of the stimuli, we can evaluate the reliable shared variance between the
two representations across *stimuli*.

Specifically, we can divide the images into two: a training split and a
test split. We can compute singular vectors on the training split, and
evalute *test* singular values on the test split, analogous to the
CV-PCA procedure:

$$
\begin{align*}
    \text{cov}(X_\text{train}, Y_\text{train})
    &= X_\text{train}^\top Y_\text{train} / (n - 1)\\
    &= U \Sigma V^\top\\
    \\
    \Sigma_\text{cross-validated}
    &= \text{cov}(X_\text{train} U, Y_\text{test} V)\\
    &= \left( X_\text{train} U \right) ^\top \left( Y_\text{test} V \right) / (n - 1)
\end{align*}
$$

## Comparing brains and DNNs

In [ ]:
# subject_1 = average_data_across_repetitions(load_dataset(subject=0, roi="general"))
# subject_2 = average_data_across_repetitions(load_dataset(subject=1, roi="general"))
# subject_1 = subject_1.sortby("stimulus_id")
# subject_2 = subject_2.sortby("stimulus_id")

# display(subject_1)
# display(subject_2)

# plssvd = PLSSVD()
# plssvd.fit(
#     data_repetition_1.values,
#     data_repetition_1.values,
# )
# rep_1_transformed = plssvd.transform(data_repetition_1.values, direction="left")
# rep_2_transformed = plssvd.transform(data_repetition_2.values, direction="right")


# n_neurons_1 = rep_1_transformed.shape[-1]
# cv_spectrum = np.diag(
#     np.cov(rep_1_transformed, rep_2_transformed, rowvar=False)[
#         :n_neurons_1, n_neurons_1:
#     ]
# )
# cv_spectrum = pd.DataFrame(
#     cv_spectrum, columns=["cross-validated singular value"]
# ).assign(rank=np.arange(len(cv_spectrum)) + 1)
# cv_spectrum = cv_spectrum.assign(
#     bin=bin_logarithmically(
#         cv_spectrum["cross-validated singular value"],
#         points_per_bin=5,
#         min_=1,
#         max_=10_000,
#     )
# )

# fig, ax = plt.subplots()
# sns.lineplot(ax=ax, data=cv_spectrum, x="rank", y="cross-validated singular value")
# sns.despine(ax=ax, offset=20)

# fig, ax = plt.subplots()
# sns.lineplot(ax=ax, data=cv_spectrum, x="rank", y="cross-validated singular value")
# ax.set_xscale("log")
# ax.set_yscale("log")
# ax.set_ylim(bottom=1e-3)
# sns.despine(ax=ax, offset=20)

# fig, ax = plt.subplots()
# sns.lineplot(
#     ax=ax,
#     data=cv_spectrum.groupby("bin").mean().rename(columns={"bin": "rank"}),
#     x="rank",
#     y="cross-validated singular value",
#     ls=None,
#     marker="o",
# )
# ax.set_xscale("log")
# ax.set_yscale("log")
# sns.despine(ax=ax, offset=20)
